## Load the Italian-Dutch parallel corpus

In [1]:
import pandas as pd
import numpy as np

np.random.seed(2)

columns = ['it_id', 'italian', 'nl_id', 'dutch']
df = pd.read_csv('Datasets/Tatoeba-it-nl.tsv', sep='\t', header=None)
df.columns = columns
df.head()

,it_id,italian,nl_id,dutch
0,4369,Devo andare a dormire.,5966,Ik moet gaan slapen.
1,4371,Che cos'è?,5970,Wat is dat?
2,4373,"La parola d'accesso è ""Muiriel"".",5985,"Het wachtwoord is ""Muiriel""."
3,4375,Non cambierà niente.,379556,Dat zal niets aan de zaak veranderen.
4,4376,Costerà trenta euro.,378907,"Dat zal € 30,- kosten."


## Define function to count the phenomena frequencies in the dataset

In [2]:
import stanza

# Load the Italian language model in stanza
stanza.download('it')
nlp = stanza.Pipeline(lang='it', processors='tokenize,mwt,pos,lemma')

# Helper functions for phenomena detection
def has_subject_omission(sentence):
    """Check if the first word of the sentence is a verb."""
    if not isinstance(sentence, str):
        return False
    doc = nlp(sentence)
    first_word_pos = doc.sentences[0].words[0].upos if doc.sentences and doc.sentences[0].words else None
    return first_word_pos == "VERB" or first_word_pos == "AUX"

def has_reflexive_construction(sentence):
    """Check if the sentence contains reflexive pronouns."""
    reflexive_pronouns = ["mi", "ti", "si", "ci", "vi"]
    return any(pronoun in sentence.split() for pronoun in reflexive_pronouns)

def has_double_negation(sentence):
    """Check if the sentence contains double negation."""
    negative_words = ["niente", "nessuno", "nessuna", "nessun", "nulla"]
    words = sentence.lower().split()
    return words.count("non") > 1 or "non" in words and any(word in words for word in negative_words)

with open("./Datasets/Diminutives-Augmentatives.txt", 'r', encoding='utf-8') as file:
    diminutives_augmentatives = [line.strip() for line in file]

def has_diminutives_or_augmentatives(sentence):
    """Check if the sentence contains diminutives or augmentatives."""
    suffixes = ["ino", "ina", "etto", "etta", "accio", "accia", "one", "ona"]
    sentence_no_punct = sentence.translate(str.maketrans("", "", string.punctuation))
    return any(word for word in sentence_no_punct.split() if word in diminutives_augmentatives)

def has_clitic_pronouns(sentence):
    """Check if the sentence contains clitic pronouns."""
    clitic_pronouns = ["lo", "la", "li", "gli", "le", "ne", "ci", "mi", "ti",
    "glielo", "gliela", "glieli", "gliele"]
    return any(pronoun in sentence.split() for pronoun in clitic_pronouns)

def phenomena_counter(df):
  phenomena_counts = {
    "Rows": len(df),
    "Sentences": len(df['it_id'].unique()),
    "Subject Omission": int(df['subject_omission'].sum()),
    "Reflexive Construction": int(df['reflexive_construction'].sum()),
    "Double Negation": int(df['double_negation'].sum()),
    "Diminutives and Augmentatives": int(df['diminutives_augmentatives'].sum()),
    "Clitic Pronouns": int(df['clitic_pronouns'].sum()),
    "Other": len(df[~(df[['subject_omission', 'reflexive_construction', 'double_negation',
                          'diminutives_augmentatives', 'clitic_pronouns']].any(axis=1))])
  }

  return phenomena_counts

2024-12-02 00:23:09 INFO: Downloaded file to C:\Users\fabia\stanza_resources\resources.json
2024-12-02 00:23:09 INFO: Downloading default packages for language: it (Italian) ...
2024-12-02 00:23:11 INFO: File exists: C:\Users\fabia\stanza_resources\it\default.zip
2024-12-02 00:23:14 INFO: Finished downloading models and saved to C:\Users\fabia\stanza_resources
2024-12-02 00:23:14 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-12-02 00:23:15 INFO: Downloaded file to C:\Users\fabia\stanza_resources\resources.json
2024-12-02 00:23:15 INFO: Loading these models for language: it (Italian):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |

2024-12-02 00:23:15 INFO: Using device: cpu
2024-12-02 00:23:15 INFO: Loading: tokenize
C:\Users\fabia\AppData\Local\Programs\Python\Python312\Lib\site-packages\stanza\models\tokenization\trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `T

In [3]:
from tqdm import tqdm

# Initialize tqdm progress bar
tqdm.pandas()

# Update functions to use tqdm for progress tracking
df['subject_omission'] = df['italian'].progress_apply(has_subject_omission)
df['reflexive_construction'] = df['italian'].progress_apply(has_reflexive_construction)
df['double_negation'] = df['italian'].progress_apply(has_double_negation)
df['diminutives_augmentatives'] = df['italian'].progress_apply(has_diminutives_or_augmentatives)
df['clitic_pronouns'] = df['italian'].progress_apply(has_clitic_pronouns)

df.to_csv('Datasets/Analysis-IT-NL.tsv', sep='\t', index=False)

 77%|███████▋  | 13481/17525 [16:01<04:48, 14.02it/s] 


KeyboardInterrupt: 

In [ ]:
phenomena_counter(df)

## Check Diminutives and Augmentatives

In [ ]:
diminutives_augmentatives = [
    word
    for words in df[df['diminutives_augmentatives']]['italian'].str.split()
    for word in words
    if word.endswith("ino") or word.endswith("etto") or word.endswith("accio") or word.endswith("one")
]

set(diminutives_augmentatives)

### Adjusting the flag based on the actual diminutives and augmentatives

In [ ]:
actual_diminutives_augmentatives = ["L'amichetto", 'laghetto', 'modellino', 'motorino', 'orsetto', 'ragazzino']

df.loc[
    df['diminutives_augmentatives'] & ~df['italian'].str.contains('|'.join(actual_diminutives_augmentatives), na=False),
    'diminutives_augmentatives'
] = False
phenomena_counter(df)

## Split the dataset into single and multiple translations

In [ ]:
multiple_translation_df = df[df['nl_id'].isin(df['nl_id'].value_counts()[lambda x: x > 1].index)]
phenomena_counter(multiple_translation_df)

In [ ]:
df = df[~df.index.isin(multiple_translation_df.index)]
phenomena_counter(df)

## Create a curated dataset with a balanced distribution of phenomena

In [ ]:
def sample_and_remove(source_df, target_df, column_name, n):
    if len(source_df[source_df[column_name]]) < n:
        n = len(source_df[source_df[column_name]])
    sample = source_df[source_df[column_name]].sample(n=n, replace=False)
    source_df = source_df.drop(sample.index)
    target_df = pd.concat([target_df, sample], ignore_index=True)
    return source_df, target_df, n

curated_df = pd.DataFrame(columns=df.columns)

phenomena = [
    'subject_omission',
    'reflexive_construction',
    'double_negation',
    'diminutives_augmentatives',
    'clitic_pronouns'
]

for phenomenon in phenomena:
    multiple_translation_df, curated_df, n = sample_and_remove(multiple_translation_df, curated_df, phenomenon, 60)
    df, curated_df, m = sample_and_remove(df, curated_df, phenomenon, 120 - n)
    if m + n != 120:
      print(f"Missing {120 - m - n} sentences for {phenomenon}\n")

## Add the generated sentences

In [ ]:
generated_df = pd.read_csv('Datasets/Double-Negation-Generated-IT-NL.tsv', sep='\t')
tmp = pd.read_csv('Datasets/Diminutives-Augmentatives-Generated-IT-NL.tsv', sep='\t')
generated_df = pd.concat([generated_df, tmp], ignore_index=True)

for _, row in generated_df.iterrows():
    sentence = row['italian']
    translation = row['dutch']
    
    if sentence in curated_df['italian'].values:
        it_id = curated_df.loc[curated_df['italian'] == sentence, 'it_id'].values[0]
    else:
        it_id = curated_df['it_id'].max() + 1 if not curated_df.empty else 1  
    
    if translation in curated_df['dutch'].values:
        nl_id = curated_df.loc[curated_df['dutch'] == translation, 'nl_id'].values[0]
    else:
        nl_id = curated_df['nl_id'].max() + 1 if not curated_df.empty else 1

    new_row = pd.DataFrame({'it_id': [it_id], 'italian': [sentence], 'nl_id': [nl_id], 'dutch': [translation]})
    curated_df = pd.concat([curated_df, new_row], ignore_index=True)

## Add random sentences for variety

In [ ]:
curated_df = pd.concat([curated_df, df.sample(n=400, replace=False)], ignore_index=True)
len(curated_df)

## Save the curated dataset

In [ ]:
curated_df.drop(columns=[
    'subject_omission',
    'reflexive_construction',
    'double_negation',
    'diminutives_augmentatives',
    'clitic_pronouns'
], inplace=True, errors='ignore')
curated_df.to_csv('Datasets/Curated-1000-IT-NL.tsv', sep='\t', index=False)